# Pipeline de Geração de Observações - Tic Tac Toe
**Author**: Gabriela Dellamora Paim, Bruno Carlan

**Version**: 12/04/2025

**Python Ver**: 3.13.1

## Checklist para geração das 64 observações de tabuleiros *in_progress*

### Distribuição de jogadas
- [ ] Seguir essa proporção:
  - 1 jogada: 4 observações  
  - 2 jogadas: 4 observações  
  - 3 jogadas: 8 observações  
  - 4 jogadas: 8 observações  
  - 5 jogadas: 12 observações  
  - 6 jogadas: 12 observações  
  - 7 jogadas: 16 observações  
- Total: **64 observações**

### Ordem dos jogadores
- [ ] Todas as partidas devem começar com **X**
- [ ] Validar que o número de jogadas de X é igual ou 1 a mais que o de O
- [ ] Determinar corretamente o jogador da vez com base no número de jogadas:
  - Número par → próximo jogador é X
  - Número ímpar → próximo jogador é O

### Validação do estado do jogo
- [ ] Nenhum estado pode conter um jogo já vencido
- [ ] Nenhum tabuleiro pode conter jogadas inválidas (ex: duas peças na mesma posição)
- [ ] Nenhum estado pode ter mais de 9 jogadas

### Diversidade de posições
- [ ] Evitar repetição de estados (usar hash ou set para controle)
- [ ] Garantir distribuição equilibrada entre as 9 posições do tabuleiro
- [ ] (Opcional) Limitar frequência de ocupação por posição (ex: máximo 50% em cada célula)

### Formato e reprodutibilidade
- [ ] Exportar as observações no formato combinado (array, dataframe, JSON, etc.)
- [ ] Incluir metadados por observação:
  - Número de jogadas  
  - Jogador da vez  
  - ID único (ou hash)
- [ ] Fixar seed aleatória (`random.seed(42)` ou equivalente) para reprodutibilidade
- [ ] (Opcional) Logar motivo de descarte de observações (ex: estado inválido, repetido, etc.)


In [23]:
import pandas as pd
import random
from collections import defaultdict

PATH_OLD = './data_processed.csv'
PATH_NEW = './data.csv'
PATH_ONGOING = './data_ongoing.csv'
X_WIN   = X = '1'
O_WIN   = O ='-1'
DRAW    = '0'
ONGOING = '0.5'

random.seed(42)

In [24]:
df_processed = pd.read_csv(PATH_OLD, index_col=False)
df_processed.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', 'category']
df_processed.describe()

,0,1,2,3,4,5,6,7,8,category
count,1021,1021,1021,1021,1021,1021,1021,1021,1021,1021
unique,6,6,6,6,6,6,6,6,6,4
top,x,x,x,x,x,x,x,x,x,1
freq,417,377,417,377,458,378,417,378,418,625


# Gerar dados Ongoing. Utilizar um critério de observação adequado

In [ ]:
class Board:
    '''
        Gerencia as regras relacionadas ao tabuleiro de TicTacToe.
        Permite movimentações pseudo-aleatórias dos jogadores.
    '''
    def __init__(self):
        self.board = [[0,0,0], [0,0,0], [0,0,0]]
        self.moves = [(0,0), (0,1), (0,2),
                      (1,0), (1,1), (1,2),
                      (2,0), (2,1), (2,2)]
        self.status = ONGOING

    def move(self, symbol : int):
        '''Gera uma movimentação de symbol, que pode ser X ou O.'''
        x, y = self._get_coordinates()
        if (x >= 0) and (y >= 0):
            self.board[x][y] = symbol
        self._update_status()

    def _update_status(self):
        '''Atualiza o estado atual do tabuleiro'''
        self.status = self._check_wins()

    def _check_wins(self):
        '''Verifica o estado do tabuleiro e o retorna'''
        # Diagonais
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return self.board[1][1]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != 0:
            return self.board[1][1]
        # Linhas e colunas
        for i in range(3):
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return self.board[0][i]
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return self.board[i][0]
        return ONGOING

    def _get_coordinates(self) -> tuple:
        '''Captura uma coordenada aleatória e remove das opções de movimentações possíveis.
        Retorna (-1, -1) se o tabuleiro estiver preenchido'''
        if not self.is_full():
           return self.moves.pop(random.randint(0, len(self.moves) - 1))
        return -1, -1

    def is_full(self) -> bool:
        ''' Retorna se o tabuleiro está totalmente preenchido'''
        return len(self.moves) <= 0

    def get_observation(self):
        ''' Transforma as observações 3x3 em 1x9 para salvar no dataframe'''
        flat = [cell for row in self.board for cell in row]
        flat.append(self.status)
        return flat

    def generate_64_ongoing_boards():
        total_required = {
            1: 4, 2: 4, 3: 8, 4: 8,
            5: 12, 6: 12, 7: 16
        }

        generated = defaultdict(list)
        seen_hashes = set()

        def board_hash(board):
            return tuple(cell for row in board for cell in row)

        def next_player(num_moves):
            return 1 if num_moves % 2 == 0 else -1

        def count_moves(board):
            flat = [cell for row in board for cell in row]
            return flat.count(1) + flat.count(-1)

        while sum(len(v) for v in generated.values()) < 64:
            board = Board()
            moves_done = 0

            while board.status == ONGOING and moves_done < 9:
                symbol = next_player(moves_done)
                board.move(symbol)
                moves_done += 1

                if board.status != ONGOING:
                    break

                flat_board = board.get_observation()[:-1]
                current_moves = count_moves(board.board)

                if current_moves in total_required and len(generated[current_moves]) < total_required[current_moves]:
                    h = board_hash(board.board)
                    if h not in seen_hashes:
                        seen_hashes.add(h)
                        jogador_da_vez = next_player(current_moves)
                        generated[current_moves].append(flat_board + [current_moves, jogador_da_vez, str(h)])

        # Junta tudo em um DataFrame
        all_obs = []
        for lst in generated.values():
            all_obs.extend(lst)

        columns = ['0','1','2','3','4','5','6','7','8','n_jogadas','jogador_vez','id_hash']
        df_final = pd.DataFrame(all_obs, columns=columns)
        return df_final

In [ ]:
# Exporta dataset para análise
df_ongoing = Board.generate_64_ongoing_boards()
df_ongoing['category'] = ONGOING
df_ongoing.to_csv(PATH_ONGOING, index=False)

In [ ]:
# concatena os datasets para termos o nosso dataset tratado
df_ongoing.drop(columns=['n_jogadas', 'jogador_vez', 'id_hash'], inplace=True)
df = pd.concat([df_processed, df_ongoing])
df.to_csv(PATH_NEW, index=False)